In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

dr_name=[]
fl_names=[]
fl_name=[]
path=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    dr_name.append(dirname)
    fl_names.append(filenames)
    
    for filename in filenames:
        fl_name.append(filename)
        a=os.path.join(dirname, filename)
        path.append(a)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import itertools

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

In [31]:
df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [32]:
df[df['image_id']=='ISIC_0031633']

,lesion_id,image_id,dx,dx_type,age,sex,localization
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [33]:
df.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [34]:
df['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [35]:
df['dx']

0          bkl
1          bkl
2          bkl
3          bkl
4          bkl
         ...  
10010    akiec
10011    akiec
10012    akiec
10013    akiec
10014      mel
Name: dx, Length: 10015, dtype: object

In [36]:
labels_dictionary_dx_num = {'nv':0, 'mel':1, 'bkl':2, 'bcc':3, 'akiec':4, 'vasc':5, 'df':6}


In [37]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [38]:
base_skin_dir = '../input/skin-cancer-mnist-ham10000'
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}
print(type(imageid_path_dict))

<class 'dict'>


In [39]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes

print(df['path'][10],"   ",df['image_id'][10])
print(len(df['path']))


../input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0025276.jpg     ISIC_0025276
10015


In [40]:
from PIL import Image as pl
im = pl.open("../input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0025276.jpg")
im=np.array(im)
print(im.shape)

(450, 600, 3)


In [41]:
labels = [labels_dictionary_dx_num[i] for i in df['dx']]
print(len(labels))

10015


In [42]:
df.isna().sum()
df['age'].fillna((df['age'].mean()), inplace=True)
df.isna().sum().sum()


0

In [43]:
from PIL import Image
print(len(df['path']))
from PIL import Image
print(len(df['path']))
data = []

for image_path in df['path']:
    
    image = Image.open(image_path)
        
    image_resised_71_71=image.resize((71,71))
    image_resised_71_71 = np.array(image_resised_71_71)
#     image_resised_224_224=image.resize((224,224))
#     image_resised_224_224=np.array(image_resised_224_224)
#     image_resised_256_256=image.resize((256,256))
#     image_resised_100_100 = np.array(image_resised_100_100)
#     image_resised_224_224 = np.array(image_resised_224_224)
#     image_resised_256_256 = np.array(image_resised_256_256)
#     image_resised_128_128=image.resize((128,128))
#     image_resised_128_128 = np.array(image_resised_128_128)
    data.append(image_resised_71_71)

10015
10015


In [44]:
print(df['path'])

0        ../input/skin-cancer-mnist-ham10000/ham10000_i...
1        ../input/skin-cancer-mnist-ham10000/ham10000_i...
2        ../input/skin-cancer-mnist-ham10000/ham10000_i...
3        ../input/skin-cancer-mnist-ham10000/ham10000_i...
4        ../input/skin-cancer-mnist-ham10000/ham10000_i...
                               ...                        
10010    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10011    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10012    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10013    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10014    ../input/skin-cancer-mnist-ham10000/ham10000_i...
Name: path, Length: 10015, dtype: object


In [45]:
print(len(data))
print(len(labels))
print(data[0].shape)
print(data[5].shape)

10015
10015
(71, 71, 3)
(71, 71, 3)


In [46]:
data=np.array(data)
labels=np.array(labels)
print(data.shape)
print(labels.shape)

(10015, 71, 71, 3)
(10015,)


In [47]:
X_train,X_test,Y_train,Y_test = train_test_split(data, labels, test_size=0.4, random_state=49)

In [48]:
print(X_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(Y_test.shape)

(6009, 71, 71, 3)
(4006, 71, 71, 3)
(4006,)
(4006,)


In [49]:
import tensorflow as tf
from tensorflow import keras

In [45]:
# num_of_classes = 7

# # setting up the layers of Neural Network

# model = keras.Sequential([
    
#     keras.layers.Flatten(input_shape=(128,128,3)),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(num_of_classes, activation='softmax')
# ])

In [46]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Define the input image size
input_shape = (71, 71, 3)

# Preprocess the input images
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# Convert target labels to one-hot vectors
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

# Create an instance of the DenseNet-201 model
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained base model
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(7, activation='softmax'))  # num_classes is the number of output classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
batch_size = 64
epochs = 100

history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, Y_test)
)




Epoch 1/100
93/93 [==============================] - 34s 228ms/step - loss: 1.3327 - accuracy: 0.6257 - val_loss: 0.8790 - val_accuracy: 0.6877
Epoch 2/100
93/93 [==============================] - 13s 138ms/step - loss: 0.9534 - accuracy: 0.6807 - val_loss: 0.8254 - val_accuracy: 0.6935
Epoch 3/100
93/93 [==============================] - 13s 143ms/step - loss: 0.8914 - accuracy: 0.6907 - val_loss: 0.8073 - val_accuracy: 0.7004
Epoch 4/100
93/93 [==============================] - 15s 162ms/step - loss: 0.8591 - accuracy: 0.6991 - val_loss: 0.7925 - val_accuracy: 0.7034
Epoch 5/100
93/93 [==============================] - 15s 159ms/step - loss: 0.8391 - accuracy: 0.7043 - val_loss: 0.7802 - val_accuracy: 0.7092
Epoch 6/100
93/93 [==============================] - 13s 144ms/step - loss: 0.8147 - accuracy: 0.7122 - val_loss: 0.7845 - val_accuracy: 0.7009
Epoch 7/100
93/93 [==============================] - 13s 139ms/step - loss: 0.8047 - accuracy: 0.7146 - val_loss: 0.7612 - val_accuracy:

In [47]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

126/126 [==============================] - 4s 29ms/step - loss: 0.6778 - accuracy: 0.7464
Test loss: 0.6778292655944824
Test accuracy: 0.7463804483413696


In [50]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Define the input image size
input_shape = (71, 71, 3)

# Preprocess the input images
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# Convert target labels to one-hot vectors
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(71, 71, 3))

for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_size = (5, 5)
        layer.padding = 'same'

base_model.layers.pop()


num_classes = 7  # Change this to the number of classes in your task

model1 = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),  # Dropout layer with rate 0.5

    tf.keras.layers.Dense(num_classes, activation='softmax')
])
learning_rate = 0.0001  # Change this to your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [51]:
# Define data augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
batch_size = 64
epochs = 10

history = model1.fit(
    datagen.flow(X_train, Y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, Y_test)
)

Epoch 1/10
93/93 [==============================] - 139s 273ms/step - loss: 1.2968 - accuracy: 0.6131 - val_loss: 1.4249 - val_accuracy: 0.6935
Epoch 2/10
93/93 [==============================] - 21s 228ms/step - loss: 0.7644 - accuracy: 0.7408 - val_loss: 0.7233 - val_accuracy: 0.7571
Epoch 3/10
93/93 [==============================] - 21s 227ms/step - loss: 0.6339 - accuracy: 0.7733 - val_loss: 0.5932 - val_accuracy: 0.7893
Epoch 4/10
93/93 [==============================] - 22s 234ms/step - loss: 0.5519 - accuracy: 0.8032 - val_loss: 0.5725 - val_accuracy: 0.7956
Epoch 5/10
93/93 [==============================] - 22s 240ms/step - loss: 0.5096 - accuracy: 0.8153 - val_loss: 0.5907 - val_accuracy: 0.7903
Epoch 6/10
93/93 [==============================] - 22s 232ms/step - loss: 0.4767 - accuracy: 0.8283 - val_loss: 0.5688 - val_accuracy: 0.7953
Epoch 7/10
93/93 [==============================] - 22s 237ms/step - loss: 0.4415 - accuracy: 0.8316 - val_loss: 0.5865 - val_accuracy: 0.795

In [52]:
# Evaluate the model
loss, accuracy = model1.evaluate(X_test, Y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

126/126 [==============================] - 4s 32ms/step - loss: 0.5579 - accuracy: 0.8125
Test loss: 0.5578660368919373
Test accuracy: 0.8125312328338623
